In [1]:
import os
from os import listdir
from os.path import isfile, join
from shutil import copy2
#from tqdm import tqdm #pip install tqdm
import progressbar #pip install progressbar2
import tensorflow as tf
import numpy as np
import pandas as pd
import glob,cv2
import sys,argparse
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score

tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
def predict(filename):
    image_size=128
    num_channels=3
    images = []
    # Reading the image using OpenCV
    image = cv2.imread(filename)
    # Resizing the image to our desired size and preprocessing will be done exactly as done during training
    image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
    images.append(image)
    images = np.array(images, dtype=np.uint8)
    images = images.astype('float32')
    images = np.multiply(images, 1.0/255.0) 
    #The input to the network is of shape [None image_size image_size num_channels]. Hence we reshape.
    x_batch = images.reshape(1, image_size,image_size,num_channels)

    ## Let us restore the saved model 
    sess = tf.Session()
    # Step-1: Recreate the network graph. At this step only graph is created.
    saver = tf.train.import_meta_graph('benign-malicious-model.meta')
    # Step-2: Now let's load the weights saved using the restore method.
    saver.restore(sess, tf.train.latest_checkpoint('./'))

    # Accessing the default graph which we have restored
    graph = tf.get_default_graph()

    # Now, let's get hold of the op that we can be processed to get the output.
    # In the original network y_pred is the tensor that is the prediction of the network
    y_pred = graph.get_tensor_by_name("y_pred:0")

    ## Let's feed the images to the input placeholders
    x= graph.get_tensor_by_name("x:0") 
    y_true = graph.get_tensor_by_name("y_true:0") 
    y_test_images = np.zeros((1, 2)) 

    ### Creating the feed_dict that is required to be fed to calculate y_pred 
    feed_dict_testing = {x: x_batch, y_true: y_test_images}
    result=sess.run(y_pred, feed_dict=feed_dict_testing)
    # result is of this format [probabiliy_of_benign probability_of_malicious]
    #print(result)
    return np.argmax(result, axis=1)[0]

In [3]:
column = ['file','gt_result','jh_result']
df_jh_part = pd.DataFrame(columns=column)
jar_loc = ['testing_data/benign/',
           'testing_data/malicious/']
for loc in jar_loc:
    index = jar_loc.index(loc)
    onlyfiles = [f for f in listdir(loc) if isfile(join(loc, f))]
    #print(len(onlyfiles))
    progress = progressbar.ProgressBar()
    for file,i in zip(onlyfiles, progress(range(len(onlyfiles)))):
        df_jh_part = df_jh_part.append({'file':file,
                                        'gt_result':index,
                                        'jh_result':predict(loc+file)},
                                       ignore_index=True)

 99% (417 of 418) |##################### | Elapsed Time: 1:00:44 ETA:   0:00:12

In [4]:
df_jh_part.shape

(781, 3)

In [5]:
gt = df_jh_part['gt_result'].tolist()
jh = df_jh_part['jh_result'].tolist()

precision, recall, fscore, support = score(gt, jh)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support),'\n')

print(classification_report(gt, jh))
print('accuracy', accuracy_score(gt, jh))

precision: [0.81940701 0.85609756]
recall: [0.83746556 0.83971292]
fscore: [0.82833787 0.84782609]
support: [363 418] 

             precision    recall  f1-score   support

          0       0.82      0.84      0.83       363
          1       0.86      0.84      0.85       418

avg / total       0.84      0.84      0.84       781

accuracy 0.8386683738796414


In [6]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(testy))]
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

No Skill: ROC AUC=0.500
Logistic: ROC AUC=0.903


<Figure size 640x480 with 1 Axes>

In [11]:
lr_probs

array([9.95972685e-01, 9.97994843e-01, 5.29883066e-03, 1.52374194e-01,
       9.98333350e-01, 5.00061448e-01, 1.97631392e-02, 2.34633843e-02,
       9.82000697e-01, 7.06465180e-01, 4.16201961e-03, 7.46446600e-01,
       7.66567488e-03, 9.61958719e-01, 9.35214568e-01, 8.07653554e-01,
       1.44803886e-01, 9.56532321e-01, 1.56957507e-01, 9.44908759e-01,
       8.67007620e-01, 4.10282064e-01, 2.04815673e-02, 6.67093752e-01,
       4.26334713e-03, 9.56694469e-01, 6.92577362e-01, 9.94406759e-01,
       2.49159988e-02, 5.50542684e-01, 7.92214513e-01, 8.15242298e-01,
       8.18291856e-02, 8.23317999e-01, 1.53516867e-02, 7.98798532e-02,
       9.87551937e-01, 9.36923464e-01, 9.86339822e-01, 3.19318661e-01,
       9.82576369e-03, 3.82722802e-02, 9.83635286e-02, 9.83780221e-01,
       6.53737641e-03, 5.29530623e-01, 5.01616608e-01, 9.97760930e-01,
       8.40960809e-03, 1.42082825e-01, 2.21934800e-02, 6.47786780e-01,
       9.79115074e-01, 2.32942383e-01, 5.09205311e-02, 9.03776284e-01,
      